In [5]:
import praw
import json

# Reddit API credentials
reddit = praw.Reddit(
    client_id='B-qrtXmBJd2BgAAwmKozQg',
    client_secret='iVwh4ZpT35sQMBvye7nKaz8839LDPQ',
    user_agent='YOUR_USER_AGENT'
)

subreddit_name = 'IndianSkincareAddicts'

subreddit = reddit.subreddit(subreddit_name)

# Keyword or product name to search for
search_query = 'Love Beauty and Planet'  # Replace with the name of your product

# Fetch posts mentioning your product
search_results = subreddit.search(search_query, limit=30)

parsed_posts = []

for post in search_results:
    parsed_posts.append({
        'post_title': post.title,
        'post_permalink': post.permalink,
        'score': post.score,
        'author': str(post.author),
        'created_utc': post.created_utc,
        'num_comments': post.num_comments,
        'selftext': post.selftext  # This is the body of the post, if available
    })

# Save to JSON
output_file_path = 'product_mentions.json'
with open(output_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(parsed_posts, json_file, ensure_ascii=False, indent=2)

print(f"Data has been saved to {output_file_path}")

Data has been saved to product_mentions.json
